In [1]:
%pylab inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import datetime as dt
from osgeo import gdal, gdal_array

Populating the interactive namespace from numpy and matplotlib


In [2]:
import yatsm
from yatsm.io import read_pixel_timeseries
from yatsm.utils import csvfile_to_dataframe, get_image_IDs
from yatsm.config_parser import convert_config, parse_config_file
from yatsm.config_parser import convert_config, parse_config_file
import yatsm._cyprep as cyprep

In [3]:
# Define image reading function
def read_image(f):
    ds = gdal.Open(f, gdal.GA_ReadOnly)
    return ds.GetRasterBand(1).ReadAsArray()

In [4]:
example_img_fn = '/projectnb/landsat/projects/Massachusetts/Broadmoor_small/images/example_img'
example_img = read_image(example_img_fn)
print('Shape of example image:')
print(example_img.shape)

Shape of example image:
(51, 77)


In [5]:
## SPECIFY CONFIG FILE
config_file = '/projectnb/landsat/projects/Massachusetts/Broadmoor_small/Broadmoor_config_pixel.yaml'

# Read in and parse config file
cfg = parse_config_file(config_file)

In [6]:
# Get files list
df = csvfile_to_dataframe(cfg['dataset']['input_file'], \
                          date_format=cfg['dataset']['date_format'])

In [69]:
disturbances.shape

(51, 77, 29)

In [91]:
site = 'BM'

# Parameters
T_TCB_diff = -500
T_TCB_sum = -500

T_TCG_veg = 0.75   # % reduction in TCG
T_TCG_open = 750

py_dim = example_img.shape[0]
px_dim = example_img.shape[1]

disturbances = np.zeros((py_dim, px_dim, 29))
veg_cond = np.zeros((py_dim, px_dim, 29))

for py in list(range(0, py_dim)): # row iterator
    print('working on row {py}'.format(py=py))
    
    for px in list(range(0, px_dim)): # column iterator
        #print(px)
        # Get dates
        df['image_ID'] = get_image_IDs(df['filename']) 
        df['x'] = df['date'] 
        dates = df['date'].values

        # Read in time series as numpy array
        Y = read_pixel_timeseries(df['filename'], px, py)
        
        if np.all(Y[0] == -9999): 
            pass
        else:
            # Mask based on physical constraints and Fmask 
            valid = cyprep.get_valid_mask( \
                        Y, \
                        cfg['dataset']['min_values'], \
                        cfg['dataset']['max_values']).astype(bool)

            # Apply mask band
            idx_mask = cfg['dataset']['mask_band'] - 1
            valid *= np.in1d(Y.take(idx_mask, axis=0), \
                                     cfg['dataset']['mask_values'], \
                                     invert=True).astype(np.bool)

            # mask time series using fmask result
            Y_fmask = np.delete(Y, idx_mask, axis=0)[:, valid]
            dates_fmask = dates[valid]

            # multitemporal mask - original time series (no fmask)
            multitemp1 = np.where(Y[1] < (np.mean(Y_fmask[1])+np.std(Y_fmask[1])*3))
            dates_multi = dates[multitemp1[0]] # mask where green > 3 stddev
            Y_multi = Y[:, multitemp1[0]]
            multitemp2 = np.where(Y_multi[4] > (np.mean(Y_fmask[4])-np.std(Y_fmask[4])*3))
            dates_multi = dates_multi[multitemp2[0]] # mask where swir < 3 std dev
            Y_multi = Y_multi[:, multitemp2[0]]

            # multi-temporal mask - fmask time series
            multitemp1_fmask = np.where(Y_fmask[1] < (np.mean(Y_fmask[1])+np.std(Y_fmask[1])*3))
            dates_fmask = dates_fmask[multitemp1_fmask[0]] # mask where green > 3 stddev
            Y_fmask = Y_fmask[:, multitemp1_fmask[0]]
            multitemp2_fmask = np.where(Y_fmask[4] > (np.mean(Y_fmask[4])-np.std(Y_fmask[4])*3))
            dates_fmask = dates_fmask[multitemp2_fmask[0]] # mask where swir < 3 std dev
            Y_fmask = Y_fmask[:, multitemp2_fmask[0]]

            # convert time from ordinal to dates
            dt_dates_multi = np.array([dt.datetime.fromordinal(d) for d in dates_multi])
            dt_dates_fmask = np.array([dt.datetime.fromordinal(d) for d in dates_fmask])

            # Create dataframes for analysis
            # Fmasked + multitemporal masked data (for TCB TS)
            shp_ = dt_dates_fmask.shape[0]
            dt_dates_fmask_csv = dt_dates_fmask.reshape(shp_, 1)
            Y_fmask_csv = np.transpose(Y_fmask)
            data_fmask = np.concatenate([dt_dates_fmask_csv, Y_fmask_csv], axis=1)
            col_names = ['date', 'blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'therm', 'tcb', 'tcg', 'tcw']
            band_names = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'therm', 'tcb', 'tcg', 'tcw']
            data_fmask_df = pd.DataFrame(data_fmask, columns=col_names)
            data_fmask_df[band_names] = data_fmask_df[band_names].astype(int) # convert reflectance to int
            year_group_fmask = data_fmask_df.groupby(data_fmask_df.date.dt.year) # annual time series
            years_fmask = np.asarray(year_group_fmask.groups.keys()) # years in time series
            years_fmask = years_fmask.astype(int)

            # Multitemporal only masked data (for TCG TS)
            shp_ = dt_dates_multi.shape[0]
            dt_dates_multi_csv = dt_dates_multi.reshape(shp_, 1)
            Y_multi_csv = np.transpose(Y_multi)
            data_multi = np.concatenate([dt_dates_multi_csv, Y_multi_csv], axis=1)
            col_names = ['date', 'blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'therm', 'tcb', 'tcg', 'tcw', 'fmask']
            band_names = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'therm', 'tcb', 'tcg', 'tcw']
            data_multi_df = pd.DataFrame(data_multi, columns=col_names)
            data_multi_df[band_names] = data_multi_df[band_names].astype(int) # convert reflectance to int
            year_group_multi = data_multi_df.groupby(data_multi_df.date.dt.year) # annual time series
            years_multi = np.asarray(year_group_multi.groups.keys()) # years in time series
            years_multi = years_multi.astype(int)

            # TC Brightness Change Detection - Flood
            TCB_mean = year_group_fmask['tcb'].mean()
            TCB_mean_diff = np.diff(TCB_mean)
            TCB_mean_sum = np.cumsum(TCB_mean_diff)
            #flood_TCB_diff = years_fmask[np.where(TCB_mean_diff < T_TCB_diff)]
            #flood_TCB_sum = years_fmask[np.where(TCB_mean_sum < T_TCB_sum)]

            # TC Greenness Change Detection - Vegetation
            TCG_min = year_group_multi['tcg'].min()
            TCG_max = year_group_multi['tcg'].max()           
            TCG_amp = np.asarray(TCG_max - TCG_min)
            TCG_amp_adj = TCG_amp.astype(float) / TCG_amp[0].astype(float) # scale by first values (assume forest conditions)
            #flood_TCG_veg = years_multi[np.where(TCG_amp_adj < T_TCG_veg)]
            #flood_TCG_open = years_multi[np.where(TCG_amp < T_TCG_open)]

            length = (years_fmask.size - 1)
            for index, year in enumerate(years_fmask): 
                if index < length:
                    if (((TCB_mean_diff[index] < T_TCB_diff) and (TCG_amp_adj[index+1] < T_TCG_veg)) or \
                         ((TCB_mean_sum[index] < T_TCB_sum) and (TCG_amp_adj[index+1] < T_TCG_veg))):
                        disturbances[py, px, index] = year
                        #print('{row}, {col} - {year}'.format(row=py, col=px, year=year))
                    else:
                        continue

            for index, year in enumerate(years_fmask):
                if index < length:
                    if (TCG_amp[index] < T_TCG_open): 
                        veg_cond[py, px, index] = 1
                    elif (TCG_amp_adj[index] > T_TCG_veg and TCG_amp_adj[index] <= 1:   
                        veg_cond[py, px, index] = 2
                    else:
                        veg_cond[py, px, index] = 3
                else:
                    continue          


working on row 0
working on row 1
working on row 2
working on row 3
working on row 4
working on row 5
working on row 6
working on row 7
working on row 8
working on row 9
working on row 10
working on row 11
working on row 12
working on row 13
working on row 14
working on row 15
working on row 16
working on row 17
working on row 18
working on row 19
working on row 20
working on row 21
working on row 22
working on row 23
working on row 24
working on row 25
working on row 26
working on row 27
working on row 28
working on row 29
working on row 30
working on row 31
working on row 32
working on row 33
working on row 34
working on row 35
working on row 36
working on row 37
working on row 38
working on row 39
working on row 40
working on row 41
working on row 42
working on row 43
working on row 44
working on row 45
working on row 46
working on row 47
working on row 48
working on row 49
working on row 50


In [92]:
in_ds = gdal.Open(example_img_fn, gdal.GA_ReadOnly)

# Output disturbance map for each year
for index, year in enumerate(years_fmask):
    if index < length:
        disturbance_fn = './{year}_disturbance.tif'.format(year=year)

        out_driver = gdal.GetDriverByName("GTiff")
        out_ds = out_driver.Create(disturbance_fn, 
                                   example_img.shape[1],  # x size
                                   example_img.shape[0],  # y size
                                   1,  # number of bands
                                   gdal.GDT_UInt32)
        out_ds.SetProjection(in_ds.GetProjection())
        out_ds.SetGeoTransform(in_ds.GetGeoTransform())
        out_ds.GetRasterBand(1).WriteArray(disturbances[:, :, index])
        out_ds.GetRasterBand(1).SetNoDataValue(0)
        out_ds.GetRasterBand(1).SetDescription('Beaver Disturbances')
        out_ds = None

In [95]:
# Output disturbance map for each year
for index, year in enumerate(years_fmask):
    if index < length:
        disturbance_fn = './{year}_vegetation.tif'.format(year=year)

        out_driver = gdal.GetDriverByName("GTiff")
        out_ds = out_driver.Create(disturbance_fn, 
                                   example_img.shape[1],  # x size
                                   example_img.shape[0],  # y size
                                   1,  # number of bands
                                   gdal.GDT_UInt32)
        out_ds.SetProjection(in_ds.GetProjection())
        out_ds.SetGeoTransform(in_ds.GetGeoTransform())
        out_ds.GetRasterBand(1).WriteArray(veg_cond[:, :, index])
        out_ds.GetRasterBand(1).SetNoDataValue(0)
        out_ds.GetRasterBand(1).SetDescription('Vegetation condition')
        out_ds = None